In [102]:
import pandas as pd
import logging
import numpy as np
from icecream import ic
from itertools import product
from geopy.distance import geodesic


logging.basicConfig(level=logging.DEBUG)

In [103]:
PATH = "./cities/"
INSTANCE = "italy.csv"

In [ ]:
cities = pd.read_csv(f"{PATH}{INSTANCE}", header=None, names=["City", "lat", "long"])
#cities

,City,lat,long
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34
5,Bolzano,46.50,11.35
6,Brescia,45.55,10.22
7,Cagliari,39.22,9.10
8,Catania,37.50,15.08
9,Ferrara,44.84,11.61


In [ ]:
#show only first column
cities_names = np.array([c['City'] for _,c in cities.iterrows()])
#cities_names

array(['Ancona', 'Andria', 'Bari', 'Bergamo', 'Bologna', 'Bolzano',
       'Brescia', 'Cagliari', 'Catania', 'Ferrara', 'Florence', 'Foggia',
       'Forlì', 'Genoa', 'Giugliano in Campania', 'Latina', 'Leghorn',
       'Messina', 'Milan', 'Modena', 'Monza', 'Naples', 'Novara', 'Padua',
       'Palermo', 'Parma', 'Perugia', 'Pescara', 'Piacenza', 'Prato',
       'Ravenna', 'Reggio di Calabria', "Reggio nell'Emilia", 'Rimini',
       'Rome', 'Salerno', 'Sassari', 'Syracuse', 'Taranto', 'Terni',
       'Trento', 'Trieste', 'Turin', 'Venice', 'Verona', 'Vicenza'],
      dtype='<U21')

In [ ]:
coordinates = cities[["lat","long"]].to_numpy()
#coordinates

array([[43.6 , 13.5 ],
       [41.23, 16.29],
       [41.12, 16.87],
       [45.7 ,  9.67],
       [44.5 , 11.34],
       [46.5 , 11.35],
       [45.55, 10.22],
       [39.22,  9.1 ],
       [37.5 , 15.08],
       [44.84, 11.61],
       [43.78, 11.24],
       [41.47, 15.55],
       [44.22, 12.03],
       [44.42,  8.93],
       [40.93, 14.19],
       [41.47, 12.89],
       [43.55, 10.3 ],
       [38.19, 15.55],
       [45.47,  9.17],
       [44.65, 10.92],
       [45.58,  9.27],
       [40.85, 14.27],
       [45.45,  8.62],
       [45.41, 11.87],
       [38.12, 13.36],
       [44.81, 10.32],
       [43.11, 12.39],
       [42.46, 14.21],
       [45.06,  9.68],
       [43.89, 11.09],
       [44.42, 12.21],
       [38.11, 15.65],
       [44.71, 10.63],
       [44.06, 12.57],
       [41.89, 12.5 ],
       [40.68, 14.77],
       [40.73,  8.56],
       [37.07, 15.29],
       [40.48, 17.24],
       [42.57, 12.65],
       [46.08, 11.12],
       [45.65, 13.77],
       [45.08,  7.68],
       [45.

## Helper funciton

In [107]:

def distance(c1,c2):
    return geodesic(c1,c2).km

#distance(coordinates[0],coordinates[1])


In [ ]:
dist_matrix = np.array([[distance(c1,c2) for c1 in coordinates] for c2 in coordinates])

#

array([[  0.        , 349.30401144, 391.04186819, ..., 223.61470849,
        285.67481712, 266.79616501],
       [349.30401144,   0.        ,  50.17877215, ..., 566.29615596,
        634.93310475, 614.95900451],
       [391.04186819,  50.17877215,   0.        , ..., 604.01577185,
        676.49916849, 654.72206469],
       ...,
       [223.61470849, 566.29615596, 604.01577185, ...,   0.        ,
        104.8569216 ,  63.17924401],
       [285.67481712, 634.93310475, 676.49916849, ..., 104.8569216 ,
          0.        ,  44.69517255],
       [266.79616501, 614.95900451, 654.72206469, ...,  63.17924401,
         44.69517255,   0.        ]])

In [109]:
def greedy_tsp():
    visited = [False]*len(coordinates)
    dist = dist_matrix.copy()

    city = 0
    tsp = list()
    #ic(dist)
    tsp.append(city)

    while not np.all(visited):
        min_dist = np.inf
        next_city = None
        for i in range(len(coordinates)):
            if not visited[i] and dist[city][i] < min_dist and i != city:
                next_city = i
                min_dist = dist[city][i]
        visited[next_city] = True
        tsp.append(next_city)
        #ic("visiting city: ", cities_names[city])
        logging.debug(
            f"step: {cities_names[city]} -> {cities_names[next_city]} ({min_dist:.2f}km)"
        )
        city = next_city
        
                

    tsp
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")  
    return tsp

## gready + hill climber

In [ ]:
initial_sol = greedy_tsp()
#initial_sol

DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

[0,
 33,
 12,
 30,
 9,
 4,
 19,
 32,
 25,
 28,
 18,
 20,
 3,
 6,
 44,
 45,
 23,
 43,
 41,
 5,
 40,
 22,
 42,
 13,
 16,
 29,
 10,
 26,
 39,
 34,
 15,
 14,
 21,
 35,
 11,
 1,
 2,
 38,
 17,
 31,
 8,
 37,
 24,
 7,
 36,
 27,
 0]